# Notebook for the experiments
In this notebook are contained the following features:
* GRAFF + Link prediction,

The main tools that have been exploited are [PyTorch](https://pytorch.org/) (1.13.0), [PyTorch-Lightning](https://www.pytorchlightning.ai/index.html) (1.5.10), [Pytorch-geometric](https://pytorch-geometric.readthedocs.io/en/latest/install/installation.html) (2.3.0) and [Weights & Biases](https://wandb.ai/)

### Requirements to run the notebook

In [1]:
# !pip install torch==1.12.1+cu113 torchvision==0.13.1+cu113 torchaudio==0.12.1 --extra-index-url https://download.pytorch.org/whl/cu113
# !pip install pytorch-lightning==1.5.10
# !pip install pyg_lib torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-1.12.0+cu113.html
# !pip install torch_geometric
# !pip install wandb

## Importing the libraries

In [2]:
######## IMPORT EXTERNAL FILES ###########
import torch
import torch.nn.functional as F
import torch.nn.utils.parametrize as parametrize
import torch.nn as nn

import torch_geometric
from torch_geometric.loader import DataLoader
from torch_geometric.transforms import RandomLinkSplit

import pytorch_lightning as pl
from pytorch_lightning.callbacks.early_stopping import EarlyStopping
from pytorch_lightning.callbacks import Callback
from pytorch_lightning.loggers import WandbLogger

import wandb

######### IMPORT INTERNAL FILES ###########
import sys
sys.path.append("../../src")
from GRAFF import *
from config import *

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Link prediction features initialized.....


### System configuration

In [3]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
num_gpus = 1 if device == 'cuda' else 0

if wb:
    wandb.login()

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: difra00 (deepl_wizards). Use `wandb login --relogin` to force relogin


## PyTorch Lightning DataModule (Link Prediction)

In [4]:
class DataModuleLP(pl.LightningDataModule):

    def __init__(self,  train_set, val_set, test_set, neg_edges, mode, batch_size):

        self.mode = mode  # "hp" or "test"
        self.batch_size = batch_size
        self.train_set, self.val_set, self.test_set = train_set, val_set, test_set
        self.neg_edges = neg_edges

    def setup(self, stage=None):
        if stage == 'fit':
            if self.mode == 'test':
                # For the test phase, after the hp tuning we unify train and val.
                self.train_set.edge_index = torch.concat((self.train_set.edge_index, self.val_set.edge_label_index), dim = -1)

            train_mask_edg = 0.5 * self.train_set.edge_index.shape[1] 

            self.train_set.pos_forward_pass = self.train_set.edge_index[:, :int(train_mask_edg)]

            # The remaining (30%) is used for the prediction
            self.train_set.pos_masked_edges = self.train_set.edge_index[:, int(train_mask_edg):]
            # The same amount used as positive in the prediction is taken from the negatives
            self.train_set.neg_edges = self.neg_edges[:, :self.train_set.pos_masked_edges.shape[1]]


        elif stage == 'test':
            # During the inference we attempt to predict the whole set as true.
            if self.mode == 'hp':
                self.val_set.neg_edges = self.neg_edges[:, self.train_set.pos_masked_edges.shape[1]: self.train_set.pos_masked_edges.shape[1] + 
                                                                    self.val_set.edge_label_index.shape[1]]
            elif self.mode == 'test':
                self.test_set.neg_edges = self.neg_edges[:, self.train_set.pos_masked_edges.shape[1]:self.train_set.pos_masked_edges.shape[1]+
                                                            self.test_set.edge_label_index.shape[1]]

    def train_dataloader(self, *args, **kwargs):
        return DataLoader([self.train_set], batch_size = batch_size, shuffle = False)
    def val_dataloader(self, *args, **kwargs):
        if self.mode == 'hp':
            return DataLoader([self.val_set], batch_size = batch_size, shuffle = False)
        elif self.mode == 'test':
            return DataLoader([self.test_set], batch_size = batch_size, shuffle = False)


In [5]:
train_data = torch.load(dataset_name + "/train_data.pt")
val_data = torch.load(dataset_name + "/val_data.pt")
test_data = torch.load(dataset_name + "/test_data.pt")
negative_edges = torch.load(dataset_name + "/negatives.pt")

In [6]:
mode = 'hp'  # hp: Hyperparameter selection mode
dataM = DataModuleLP(train_data.clone(), val_data.clone(), test_data.clone(), negative_edges, mode = mode, batch_size = batch_size)
dataM.setup(stage='fit')
dataM.setup(stage='test') 

### PyTorch Lightning Callbacks

In [7]:

class Get_Metrics(Callback):

    def on_train_epoch_end(self, trainer: "pl.Trainer", pl_module: "pl.LightningModule"):

        # Compute the metrics
        train_loss = sum(
            pl_module.train_prop['loss']) / len(pl_module.train_prop['loss'])
        train_acc = sum(
            pl_module.train_prop['HR@100']) / len(pl_module.train_prop['HR@100'])
        test_loss = sum(
            pl_module.test_prop['loss']) / len(pl_module.test_prop['loss'])
        test_acc = sum(pl_module.test_prop['HR@100']) / \
            len(pl_module.test_prop['HR@100'])

        # Log the metrics
        pl_module.log(name='Loss on train', value=train_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='HR@100 on train', value=train_acc,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='Loss on test', value=test_loss,
                      on_epoch=True, prog_bar=True, logger=True)
        pl_module.log(name='HR@100 on test', value=test_acc,
                      on_epoch=True, prog_bar=True, logger=True)

        # Re-initialize the metrics
        pl_module.train_prop['loss'] = []
        pl_module.train_prop['HR@100'] = []
        pl_module.test_prop['loss'] = []
        pl_module.test_prop['HR@100'] = []

## PyTorch Lightning Training Module (Node Classification)

In [8]:
class TrainingModule(pl.LightningModule):

    def __init__(self, model, lr, wd):
        super().__init__()
        self.model = model.to(device)
        self.lr = lr
        self.wd = wd

        self.train_prop = {'loss': [], 'HR@100': []}
        self.test_prop = {'loss': [], 'HR@100': []}
       

    def training_step(self, batch, batch_idx):
       

        out = self.model(batch, train = True)
        acc = evaluate(out)

        pos_pred, neg_pred = out

        loss = -torch.log(pos_pred + 1e-15).mean() - torch.log(1 - neg_pred + 1e-15).mean()
        self.train_prop['loss'].append(loss)
        self.train_prop['HR@100'].append(acc)

        return loss

    def validation_step(self, batch, batch_idx):

        out = self.model(batch, train = False)
        acc = evaluate(out)

        pos_pred, neg_pred = out

        loss = -torch.log(pos_pred + 1e-15).mean() - torch.log(1 - neg_pred + 1e-15).mean()
        self.test_prop['loss'].append(loss)
        self.test_prop['HR@100'].append(acc)

        return loss

    def configure_optimizers(self):
        self.optimizer = torch.optim.Adam(
            self.model.parameters(), lr=self.lr, weight_decay=self.wd)
        return self.optimizer


def evaluate(out):
    
    # pos_pred = torch.round(out[0])
    # neg_pred = torch.round(out[1])

    # acc = ((neg_pred == 0).sum() + pos_pred.sum()) / (neg_pred.shape[0] + pos_pred.shape[0])
    k_score = min(100, out[0].shape[0])
    pred_positive = torch.topk(out[0], k_score).values

    true_pos = (pred_positive > 0.5).sum()

    hr = true_pos / k_score

    return hr

In [9]:

#### hp enables a grid search on a wide set of hyperparameters.
if mode != 'hp':
   model = PhysicsGNN_LP(dataset, hidden_dim, output_dim, num_layers, mlp_layer, link_bias, dropout)
   pl_training_module = TrainingModule(model, lr, wd)
 


### Hyperparameters Tuning

In [10]:
def sweep_train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config
        model = PhysicsGNN_LP(dataset, config.hidden_dim, config.output_dim,
                              config.num_layers, config.mlp_layer, config.link_bias, config.dropout,
                              step=config.step)
        pl_training_module = TrainingModule(
            model, config.lr, config.wd)
        exp_name = "Sweep_LinkPred"
        wandb_logger = WandbLogger(
            project=project_name, name=exp_name, config=hyperparameters)
        trainer = trainer = pl.Trainer(
            max_epochs=epochs,  # maximum number of epochs.
            gpus=num_gpus,  # the number of gpus we have at our disposal.
            default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
            logger=wandb_logger
        )
        trainer.fit(model=pl_training_module, datamodule=dataM)


if mode == 'hp':

    if wb == False:
        if wb == False:
            model = PhysicsGNN_LP(dataset, hidden_dim, output_dim, num_layers, mlp_layer, link_bias, dropout)
            pl_training_module = TrainingModule(model, lr, wd)
    else:

    
        import pprint

        pprint.pprint(sweep_config)

        sweep_id = wandb.sweep(sweep_config, project=project_name)

        wandb.agent(sweep_id, sweep_train, count=500)

        wandb.finish()

{'method': 'random',
 'metric': {'goal': 'minimize', 'name': 'Loss on test'},
 'parameters': {'dropout': {'values': [0, 0.2, 0.4]},
                'hidden_dim': {'values': [32, 64, 128, 256, 512, 1024]},
                'link_bias': {'values': [True, False]},
                'lr': {'values': [0.001, 0.0001, 1e-05]},
                'mlp_layer': {'values': [0, 1, 2, 3]},
                'num_layers': {'values': [1, 2, 3, 4]},
                'output_dim': {'values': [16, 32, 64]},
                'step': {'values': [0.1, 0.2, 0.3, 0.4, 0.5]},
                'wd': {'values': [0, 0.01, 0.0001, 1e-06]}}}
Create sweep with ID: yxi98358
Sweep URL: https://wandb.ai/deepl_wizards/Link%20Prediction%20with%20PBGNN/sweeps/yxi98358


wandb: Agent Starting Run: 3iwomq1q with config:
wandb: 	dropout: 0.2
wandb: 	hidden_dim: 64
wandb: 	link_bias: False
wandb: 	lr: 0.0001
wandb: 	mlp_layer: 2
wandb: 	num_layers: 2
wandb: 	output_dim: 32
wandb: 	step: 0.4
wandb: 	wd: 1e-06
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


GPU available: False, used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.setup has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.setup.
  rank_zero_deprecation(

  | Name  | Type          | Params
----------------------------------------
0 | model | PhysicsGNN_LP | 117 K 
----------------------------------------
117 K     Trainable params
0         Non-trainable params
117 K     Total params
0.470     Total estimated model params size (MB)
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/callbacks/model_checkpoint.py:631: UserWarning: Checkpoint directory /home/peppe/Desktop/Università/magistrale/Thesis/Link_Prediction_with_Physics-based_GNNs-/src/link_prediction/Sweep_LinkPred/version_None/checkpoints exists

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, val_dataloader 0, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:132: UserWarning: The dataloader, train_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 12 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/trainer/data_loading.py:432: UserWarning: The number of training samples (1) is smaller than the logging interval Trainer(log_every_n_s

Epoch 1:  50%|█████     | 1/2 [00:00<00:00, 49.45it/s, loss=1.39, v_num=mq1q, Loss on train=1.390, HR@100 on train=0.690, Loss on test=1.390, HR@100 on test=0.953]

...

/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/loggers/wandb.py:341: UserWarning: There is a wandb run already in progress and newly created instances of `WandbLogger` will reuse this run. If this is not desired, call `wandb.finish()` before instantiating `WandbLogger`.
  rank_zero_warn(


Epoch 15: 100%|██████████| 2/2 [00:00<00:00, 59.85it/s, loss=1.39, v_num=7gzh, Loss on train=1.390, HR@100 on train=0.010, Loss on test=1.390, HR@100 on test=0.000]


/home/peppe/miniconda3/envs/my_env/lib/python3.9/site-packages/pytorch_lightning/core/datamodule.py:469: LightningDeprecationWarning: DataModule.teardown has already been called, so it will not be called again. In v1.6 this behavior will change to always call DataModule.teardown.
  rank_zero_deprecation(


HR@100 on test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
HR@100 on train,▄▃▃▃▁█▃▁▃▁▃▄▄▁▃▁
Loss on test,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
Loss on train,▆█▆▅▅▃▆▅▄▅▄▁▆▅▅▄
epoch,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
trainer/global_step,▁▁▂▂▃▃▄▄▅▅▆▆▇▇██
HR@100 on test,0.0
HR@100 on train,0.01
Loss on test,1.38629
Loss on train,1.38629
epoch,15


wandb: Agent Starting Run: dv0ves9u with config:
wandb: 	dropout: 0.4
wandb: 	hidden_dim: 64
wandb: 	link_bias: True
wandb: 	lr: 1e-05
wandb: 	mlp_layer: 2
wandb: 	num_layers: 3
wandb: 	output_dim: 32
wandb: 	step: 0.2
wandb: 	wd: 0
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


In [ ]:


if wb:
    exp_name = "Node_class_lr: " + \
        str(hyperparameters['learning rate']) + \
        '_wd: ' + str(hyperparameters['weight decay'])
    description = ' initial tests'
    exp_name += description
    wandb_logger = WandbLogger(
        project=project_name, name=exp_name, config=hyperparameters)


trainer = trainer = pl.Trainer(
    max_epochs=epochs,  # maximum number of epochs.
    gpus=num_gpus,  # the number of gpus we have at our disposal.
    default_root_dir="", callbacks=[Get_Metrics(), EarlyStopping('Loss on test', mode='min', patience=15)],
    logger=wandb_logger if wb else None

)

In [ ]:
trainer.fit(model = pl_training_module, datamodule = dataM)
if wb:
    wandb.finish()